In [ ]:
# Celda 1: Importaciones y configuración inicial
import numpy as np
import json
import time
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Configuración de matplotlib para mejores gráficos
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

# Celda 2: Definición de la clase de optimización
class OptimizacionNoConvexa:
    def __init__(self):
        """
        Clase para analizar y optimizar la función f(x,y) = ln²(e^x + y² + 1)
        """
        self.funcion_objetivo = self.f
        self.gradiente = self.grad_f
        
    def f(self, params):
        """
        Función objetivo: f(x,y) = ln²(e^x + y² + 1)
        """
        x, y = params
        inner = np.exp(x) + y**2 + 1
        return (np.log(inner))**2
    
    def grad_f(self, params):
        """
        Gradiente de la función objetivo
        """
        x, y = params
        inner = np.exp(x) + y**2 + 1
        log_inner = np.log(inner)
        
        df_dx = 2 * log_inner * (np.exp(x) / inner)
        df_dy = 2 * log_inner * (2 * y / inner)
        
        return np.array([df_dx, df_dy])
    
    def descenso_gradiente(self, punto_inicial, learning_rate=0.1, max_iter=1000, tol=1e-8):
        """
        Algoritmo de Descenso de Gradiente
        """
        punto_actual = np.array(punto_inicial, dtype=float)
        trayectoria = [punto_actual.copy()]
        valores_funcion = [self.f(punto_actual)]
        
        for i in range(max_iter):
            grad = self.grad_f(punto_actual)
            nuevo_punto = punto_actual - learning_rate * grad
            
            # Verificar límites del dominio [-100, 100]
            nuevo_punto = np.clip(nuevo_punto, -100, 100)
            
            trayectoria.append(nuevo_punto.copy())
            valores_funcion.append(self.f(nuevo_punto))
            
            # Criterio de parada
            if np.linalg.norm(nuevo_punto - punto_actual) < tol:
                break
                
            punto_actual = nuevo_punto
        
        resultado = {
            'punto_optimo': nuevo_punto.tolist(),
            'valor_optimo': float(self.f(nuevo_punto)),
            'iteraciones': len(trayectoria),
            'norma_gradiente': float(np.linalg.norm(grad)),
            'trayectoria': [p.tolist() for p in trayectoria],
            'valores_funcion': valores_funcion,
            'learning_rate': learning_rate
        }
        
        return resultado
    
    def bfgs_optimizacion(self, punto_inicial):
        """
        Algoritmo BFGS usando scipy.optimize
        """
        tiempo_inicio = time.time()
        
        resultado = minimize(self.funcion_objetivo, punto_inicial, 
                           method='BFGS', jac=self.gradiente,
                           options={'gtol': 1e-8, 'disp': False})
        
        tiempo_ejecucion = time.time() - tiempo_inicio
        
        resultado_dict = {
            'punto_optimo': resultado.x.tolist(),
            'valor_optimo': float(resultado.fun),
            'iteraciones': int(resultado.nit),
            'norma_gradiente': float(np.linalg.norm(resultado.jac)),
            'tiempo_ejecucion': tiempo_ejecucion,
            'exito': bool(resultado.success)
        }
        
        return resultado_dict
    
    def nelder_mead_optimizacion(self, punto_inicial):
        """
        Algoritmo Nelder-Mead usando scipy.optimize
        """
        tiempo_inicio = time.time()
        
        resultado = minimize(self.funcion_objetivo, punto_inicial, 
                           method='Nelder-Mead',
                           options={'xatol': 1e-8, 'fatol': 1e-8, 'disp': False})
        
        tiempo_ejecucion = time.time() - tiempo_inicio
        
        resultado_dict = {
            'punto_optimo': resultado.x.tolist(),
            'valor_optimo': float(resultado.fun),
            'iteraciones': int(resultado.nit),
            'tiempo_ejecucion': tiempo_ejecucion,
            'exito': bool(resultado.success)
        }
        
        return resultado_dict
    
    def experimento_completo(self, punto_inicial, learning_rates=[0.01, 0.1, 0.5]):
        """
        Ejecuta todos los algoritmos para un punto inicial dado
        """
        resultados = {
            'punto_inicial': punto_inicial,
            'descenso_gradiente': {},
            'bfgs': {},
            'nelder_mead': {}
        }
        
        # Probar diferentes learning rates para Descenso de Gradiente
        for lr in learning_rates:
            resultados['descenso_gradiente'][f'lr_{lr}'] = self.descenso_gradiente(
                punto_inicial, learning_rate=lr
            )
        
        # BFGS
        resultados['bfgs'] = self.bfgs_optimizacion(punto_inicial)
        
        # Nelder-Mead
        resultados['nelder_mead'] = self.nelder_mead_optimizacion(punto_inicial)
        
        return resultados
    
    def visualizar_funcion(self, rango_x=(-5, 5), rango_y=(-5, 5), puntos=100):
        """
        Visualización 3D de la función objetivo
        """
        x = np.linspace(rango_x[0], rango_x[1], puntos)
        y = np.linspace(rango_y[0], rango_y[1], puntos)
        X, Y = np.meshgrid(x, y)
        
        Z = np.zeros_like(X)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i,j] = self.f([X[i,j], Y[i,j]])
        
        # Crear figura con subplots
        fig = plt.figure(figsize=(18, 6))
        
        # Subplot 1: Superficie 3D
        ax1 = fig.add_subplot(131, projection='3d')
        surf = ax1.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.8)
        ax1.set_title('Función: $ln^2(e^x + y^2 + 1)$')
        ax1.set_xlabel('x')
        ax1.set_ylabel('y')
        ax1.set_zlabel('f(x,y)')
        fig.colorbar(surf, ax=ax1, shrink=0.5, aspect=5)
        
        # Subplot 2: Curvas de nivel
        ax2 = fig.add_subplot(132)
        contour = ax2.contour(X, Y, Z, levels=20)
        ax2.set_title('Curvas de Nivel')
        ax2.set_xlabel('x')
        ax2.set_ylabel('y')
        plt.colorbar(contour, ax=ax2)
        
        # Subplot 3: Heatmap
        ax3 = fig.add_subplot(133)
        im = ax3.imshow(Z, extent=[rango_x[0], rango_x[1], rango_y[0], rango_y[1]], 
                       origin='lower', cmap=cm.viridis, aspect='auto')
        ax3.set_title('Mapa de Calor')
        ax3.set_xlabel('x')
        ax3.set_ylabel('y')
        plt.colorbar(im, ax=ax3)
        
        plt.tight_layout()
        plt.show()
        
        return fig

# Celda 3: Funciones auxiliares para manejar archivos JSON
def cargar_configuracion(archivo_config='puntos_experimentos.json'):
    """
    Carga la configuración de experimentos desde un archivo JSON
    """
    try:
        with open(archivo_config, 'r') as f:
            config = json.load(f)
        return config
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {archivo_config}")
        print("Por favor, crea un archivo JSON con la siguiente estructura:")
        print("""
{
    "puntos_iniciales": [
        [0, 0],
        [10, 10],
        [50, 50],
        [-50, -50],
        [100, 0],
        [-100, 100]
    ],
    "learning_rates": [0.01, 0.1, 0.5],
    "parametros_adicionales": {
        "tolerancia": 1e-8,
        "max_iteraciones": 1000
    }
}
        """)
        return None

def guardar_resultados(resultados, archivo_salida='resultados_optimizacion.json'):
    """
    Guarda los resultados en un archivo JSON
    """
    with open(archivo_salida, 'w') as f:
        json.dump(resultados, f, indent=2)

def crear_archivo_configuracion_ejemplo():
    """
    Crea un archivo de configuración de ejemplo si no existe
    """
    config_ejemplo = {
        "puntos_iniciales": [
            [0, 0],
            [10, 10],
            [50, 50],
            [-50, -50],
            [100, 0],
            [-100, 100],
            [25, -25],
            [-75, 75]
        ],
        "learning_rates": [0.01, 0.1, 0.5],
        "parametros_adicionales": {
            "tolerancia": 1e-8,
            "max_iteraciones": 1000
        }
    }
    
    with open('puntos_experimentos.json', 'w') as f:
        json.dump(config_ejemplo, f, indent=2)
    
    print("Archivo de configuración de ejemplo creado: puntos_experimentos.json")

# Celda 4: Función principal de experimentos
def ejecutar_experimentos_desde_json(archivo_config='puntos_experimentos.json'):
    """
    Función principal que ejecuta todos los experimentos desde un archivo JSON
    """
    # Cargar configuración
    config = cargar_configuracion(archivo_config)
    if config is None:
        # Crear archivo de ejemplo y salir
        crear_archivo_configuracion_ejemplo()
        return None, None
    
    puntos_iniciales = config['puntos_iniciales']
    learning_rates = config.get('learning_rates', [0.01, 0.1, 0.5])
    
    print(f"Configuración cargada desde: {archivo_config}")
    print(f"Número de puntos iniciales: {len(puntos_iniciales)}")
    print(f"Learning rates a probar: {learning_rates}")
    
    # Inicializar el optimizador
    optimizador = OptimizacionNoConvexa()
    
    # Visualizar la función
    print("\nVisualizando la función objetivo...")
    optimizador.visualizar_funcion()
    
    # Ejecutar experimentos
    print("\nEjecutando experimentos...")
    resultados_totales = {
        'archivo_configuracion': archivo_config,
        'fecha_ejecucion': time.strftime("%Y-%m-%d %H:%M:%S"),
        'configuracion': {
            'puntos_iniciales': puntos_iniciales,
            'learning_rates': learning_rates,
            'funcion_objetivo': 'ln^2(e^x + y^2 + 1)',
            'dominio': '[-100, 100] x [-100, 100]'
        },
        'experimentos': []
    }
    
    for i, punto in enumerate(puntos_iniciales):
        print(f"\nProcesando punto inicial {i+1}/{len(puntos_iniciales)}: {punto}")
        
        resultados_punto = optimizador.experimento_completo(punto, learning_rates)
        resultados_totales['experimentos'].append(resultados_punto)
        
        # Mostrar resultados parciales
        mejor_gd = min(resultados_punto['descenso_gradiente'].values(), 
                      key=lambda x: x['valor_optimo'])
        print(f"  Mejor GD: {mejor_gd['valor_optimo']:.2e} en {mejor_gd['iteraciones']} iteraciones")
        print(f"  BFGS: {resultados_punto['bfgs']['valor_optimo']:.2e} en {resultados_punto['bfgs']['iteraciones']} iteraciones")
        print(f"  Nelder-Mead: {resultados_punto['nelder_mead']['valor_optimo']:.2e} en {resultados_punto['nelder_mead']['iteraciones']} iteraciones")
    
    # Guardar resultados
    archivo_resultados = f"resultados_optimizacion_{time.strftime('%Y%m%d_%H%M%S')}.json"
    guardar_resultados(resultados_totales, archivo_resultados)
    print(f"\nResultados guardados en: {archivo_resultados}")
    
    return resultados_totales, optimizador, archivo_resultados

# Celda 5: Ejecutar los experimentos desde JSON
print("INICIANDO EXPERIMENTOS DE OPTIMIZACIÓN DESDE ARCHIVO JSON")
print("=" * 60)

# ESPECIFICAR AQUÍ EL NOMBRE DEL ARCHIVO JSON CON LOS PUNTOS
archivo_puntos = "puntos.json"  # ← CAMBIA ESTE NOMBRE POR TU ARCHIVO

resultados_completos, optimizador, archivo_resultados = ejecutar_experimentos_desde_json(archivo_puntos)

if resultados_completos is None:
    print("No se pudieron ejecutar los experimentos. Se creó un archivo de ejemplo.")
else:
    # Celda 6: Análisis y visualización de resultados
    def analizar_resultados(archivo_resultados):
        with open(archivo_resultados, 'r') as f:
            resultados = json.load(f)
        
        # Convertir a DataFrame para análisis
        datos = []
        for i, exp in enumerate(resultados['experimentos']):
            punto = exp['punto_inicial']
            
            # Mejor Descenso de Gradiente
            mejor_gd = min(exp['descenso_gradiente'].values(), 
                          key=lambda x: x['valor_optimo'])
            
            datos.append({
                'punto_inicial': f"({punto[0]}, {punto[1]})",
                'x_inicial': punto[0],
                'y_inicial': punto[1],
                'gd_valor': mejor_gd['valor_optimo'],
                'gd_iteraciones': mejor_gd['iteraciones'],
                'gd_gradiente': mejor_gd['norma_gradiente'],
                'bfgs_valor': exp['bfgs']['valor_optimo'],
                'bfgs_iteraciones': exp['bfgs']['iteraciones'],
                'bfgs_gradiente': exp['bfgs']['norma_gradiente'],
                'bfgs_tiempo': exp['bfgs']['tiempo_ejecucion'],
                'nm_valor': exp['nelder_mead']['valor_optimo'],
                'nm_iteraciones': exp['nelder_mead']['iteraciones'],
                'nm_tiempo': exp['nelder_mead']['tiempo_ejecucion']
            })
        
        df = pd.DataFrame(datos)
        return df, resultados

    # Celda 7: Cargar resultados y mostrar resumen
    print("\nCargando resultados de optimización...")
    df, resultados_completos = analizar_resultados(archivo_resultados)

    print("Resumen de resultados:")
    print("=" * 80)
    print(df.to_string(index=False))

    # Celda 8: Análisis estadístico
    print("\n" + "=" * 80)
    print("ANÁLISIS ESTADÍSTICO")
    print("=" * 80)

    print("\nValores óptimos promedio:")
    print(f"Descenso Gradiente: {df['gd_valor'].mean():.2e} ± {df['gd_valor'].std():.2e}")
    print(f"BFGS: {df['bfgs_valor'].mean():.2e} ± {df['bfgs_valor'].std():.2e}")
    print(f"Nelder-Mead: {df['nm_valor'].mean():.2e} ± {df['nm_valor'].std():.2e}")

    print("\nIteraciones promedio:")
    print(f"Descenso Gradiente: {df['gd_iteraciones'].mean():.1f} ± {df['gd_iteraciones'].std():.1f}")
    print(f"BFGS: {df['bfgs_iteraciones'].mean():.1f} ± {df['bfgs_iteraciones'].std():.1f}")
    print(f"Nelder-Mead: {df['nm_iteraciones'].mean():.1f} ± {df['nm_iteraciones'].std():.1f}")

    # Celda 9: Visualización comparativa
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Gráfico 1: Comparación de valores óptimos
    algoritmos = ['GD', 'BFGS', 'NM']
    valores_promedio = [df['gd_valor'].mean(), df['bfgs_valor'].mean(), df['nm_valor'].mean()]
    valores_std = [df['gd_valor'].std(), df['bfgs_valor'].std(), df['nm_valor'].std()]

    axes[0,0].bar(algoritmos, valores_promedio, yerr=valores_std, capsize=5, alpha=0.7)
    axes[0,0].set_ylabel('Valor de f(x,y) (escala log)')
    axes[0,0].set_yscale('log')
    axes[0,0].set_title('Comparación de Valores Óptimos Promedio')
    axes[0,0].grid(True, alpha=0.3)

    # Gráfico 2: Comparación de iteraciones
    iter_promedio = [df['gd_iteraciones'].mean(), df['bfgs_iteraciones'].mean(), df['nm_iteraciones'].mean()]
    iter_std = [df['gd_iteraciones'].std(), df['bfgs_iteraciones'].std(), df['nm_iteraciones'].std()]

    axes[0,1].bar(algoritmos, iter_promedio, yerr=iter_std, capsize=5, alpha=0.7, color='orange')
    axes[0,1].set_ylabel('Número de Iteraciones')
    axes[0,1].set_title('Comparación de Iteraciones Promedio')
    axes[0,1].grid(True, alpha=0.3)

    # Gráfico 3: Tiempos de ejecución (solo BFGS y Nelder-Mead)
    tiempos_promedio = [df['bfgs_tiempo'].mean(), df['nm_tiempo'].mean()]
    tiempos_std = [df['bfgs_tiempo'].std(), df['nm_tiempo'].std()]

    axes[1,0].bar(['BFGS', 'Nelder-Mead'], tiempos_promedio, yerr=tiempos_std, capsize=5, alpha=0.7, color='green')
    axes[1,0].set_ylabel('Tiempo de Ejecución (s)')
    axes[1,0].set_title('Comparación de Tiempos de Ejecución')
    axes[1,0].grid(True, alpha=0.3)

    # Gráfico 4: Norma del gradiente final
    grad_promedio = [df['gd_gradiente'].mean(), df['bfgs_gradiente'].mean()]
    grad_std = [df['gd_gradiente'].std(), df['bfgs_gradiente'].std()]

    axes[1,1].bar(['GD', 'BFGS'], grad_promedio, yerr=grad_std, capsize=5, alpha=0.7, color='red')
    axes[1,1].set_ylabel('Norma del Gradiente (escala log)')
    axes[1,1].set_yscale('log')
    axes[1,1].set_title('Comparación de Norma del Gradiente Final')
    axes[1,1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Celda 10: Análisis de sensibilidad del learning rate
    print("\n" + "=" * 80)
    print("ANÁLISIS DE SENSIBILIDAD - LEARNING RATE")
    print("=" * 80)

    # Tomar el primer punto de los experimentos para el análisis de sensibilidad
    punto_prueba = resultados_completos['experimentos'][0]['punto_inicial']
    learning_rates = [0.001, 0.01, 0.1, 0.5, 1.0]

    resultados_lr = {}
    for lr in learning_rates:
        resultado = optimizador.descenso_gradiente(punto_prueba, learning_rate=lr)
        resultados_lr[lr] = resultado
        print(f"LR = {lr}: valor = {resultado['valor_optimo']:.2e}, iteraciones = {resultado['iteraciones']}")

    # Celda 11: Visualización de trayectorias
    def visualizar_trayectorias(optimizador, punto_inicial, resultados):
        """Visualiza las trayectorias de optimización"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Crear malla para fondo
        x = np.linspace(min(punto_inicial[0]-2, -2), max(punto_inicial[0]+2, 2), 50)
        y = np.linspace(min(punto_inicial[1]-2, -2), max(punto_inicial[1]+2, 2), 50)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i,j] = optimizador.f([X[i,j], Y[i,j]])
        
        # Gráfico de trayectorias
        contour = ax1.contour(X, Y, Z, levels=20, alpha=0.6)
        ax1.clabel(contour, inline=True, fontsize=8)
        
        # Trayectoria GD (usar el mejor learning rate)
        mejor_lr = min(resultados['descenso_gradiente'].keys(), 
                      key=lambda k: resultados['descenso_gradiente'][k]['valor_optimo'])
        trayectoria_gd = np.array(resultados['descenso_gradiente'][mejor_lr]['trayectoria'])
        ax1.plot(trayectoria_gd[:, 0], trayectoria_gd[:, 1], 'ro-', markersize=3, linewidth=1, label='GD')
        
        # Puntos inicial y final
        ax1.plot(punto_inicial[0], punto_inicial[1], 'go', markersize=8, label='Inicio')
        ax1.plot(trayectoria_gd[-1, 0], trayectoria_gd[-1, 1], 'bo', markersize=8, label='Final GD')
        
        ax1.set_xlabel('x')
        ax1.set_ylabel('y')
        ax1.set_title(f'Trayectorias de Optimización - Punto {punto_inicial}')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Gráfico de convergencia
        ax2.semilogy(resultados['descenso_gradiente'][mejor_lr]['valores_funcion'], 'r-', label='GD')
        ax2.set_xlabel('Iteración')
        ax2.set_ylabel('f(x,y)')
        ax2.set_title('Convergencia del Algoritmo')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

    # Visualizar trayectorias para algunos puntos
    print("\nVisualizando trayectorias para algunos puntos...")
    puntos_a_visualizar = min(3, len(resultados_completos['experimentos']))
    for i in range(puntos_a_visualizar):
        exp = resultados_completos['experimentos'][i]
        punto = exp['punto_inicial']
        print(f"Punto {i+1}: {punto}")
        visualizar_trayectorias(optimizador, punto, exp)

    # Celda 12: Exportar análisis resumido
    resumen_analisis = {
        'archivo_configuracion': archivo_puntos,
        'archivo_resultados': archivo_resultados,
        'fecha_analisis': time.strftime("%Y-%m-%d %H:%M:%S"),
        'mejor_algoritmo_global': 'BFGS' if df['bfgs_valor'].mean() < df['gd_valor'].mean() and df['bfgs_valor'].mean() < df['nm_valor'].mean() else 'GD' if df['gd_valor'].mean() < df['nm_valor'].mean() else 'NM',
        'valor_minimo_global': min(df['gd_valor'].min(), df['bfgs_valor'].min(), df['nm_valor'].min()),
        'eficiencia_promedio': {
            'GD_iteraciones': df['gd_iteraciones'].mean(),
            'BFGS_iteraciones': df['bfgs_iteraciones'].mean(),
            'NM_iteraciones': df['nm_iteraciones'].mean()
        },
        'precision_promedio': {
            'GD_gradiente': df['gd_gradiente'].mean(),
            'BFGS_gradiente': df['bfgs_gradiente'].mean()
        },
        'resumen_por_punto': df.to_dict('records')
    }

    archivo_resumen = f"analisis_resumen_{time.strftime('%Y%m%d_%H%M%S')}.json"
    with open(archivo_resumen, 'w') as f:
        json.dump(resumen_analisis, f, indent=2)

    print(f"\nAnálisis resumido guardado en: {archivo_resumen}")
    print("\nRESUMEN FINAL:")
    print(f"Mejor algoritmo global: {resumen_analisis['mejor_algoritmo_global']}")
    print(f"Valor mínimo encontrado: {resumen_analisis['valor_minimo_global']:.2e}")